In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plotting.paper import plot_trained_draws, plot_training_histories
from plotting.scores import show_all_loss_scores, show_loss_scores

# Experiments for different loss functions

In [4]:
show_all_loss_scores([(16, "exp1", 1), (16, "exp2", 1), (16, "exp3", 1)])

Frobenius norms,1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
RCL+KLD,8.281937,9.524274,8.804399,9.851355,10.423142,11.401453,12.411107,13.510895,14.652328
KLD+10mmd_rbf_sum-1.0,72.807159,45.859921,66.643036,46.035511,61.201866,47.492889,58.259453,48.569908,nan
KLD+10mmd_rbf_sum-4.0,43.161266,56.721821,53.187634,57.450008,58.103603,60.864082,62.534210,64.701546,66.370926
KLD+10mmd_rbf_sum-7.0,23.615610,50.067070,43.658802,46.838760,46.484966,48.865532,49.971104,51.678642,52.789631
KLD+10mmd_rbf_sum-10.0,28.775999,50.904537,73.190353,47.644955,61.084656,45.244717,53.457352,43.766647,48.791916
RCL+KLD+10mmd_rbf_sum-1.0,3.726709,4.362447,3.045920,4.792714,5.293283,7.255837,9.056473,11.069084,12.978620
RCL+KLD+10mmd_rbf_sum-4.0,3.737880,9.252775,6.101194,10.234677,8.580215,10.699916,10.650798,12.129404,12.781869
RCL+KLD+10mmd_rbf_sum-7.0,4.368317,10.938683,7.339455,10.754557,8.880342,10.268909,9.909635,10.869411,11.147399
RCL+KLD+10mmd_rbf_sum-10.0,4.229868,8.364177,4.605630,8.060596,6.676609,8.906524,9.928971,11.915440,13.551761


MMD scores,mmd_rbf_sum-1.0,mmd_rbf_sum-4.0,mmd_rbf_sum-10.0,"mmd_rqk_sum-4.0,0.25","mmd_rqk_sum-8.0,0.25","mmd_rqk_sum-4.0,1","mmd_rqk_sum-8.0,1","mmd_rqk_sum-4.0,10","mmd_rqk_sum-8.0,10","mmd_rqk_sum-4.0,100","mmd_rqk_sum-8.0,100"
Loss functions,,,,,,,,,,,
RCL+KLD,0.006706,0.118825,0.121253,0.064119,0.047992,0.115005,0.076634,0.149453,0.099071,0.152699,0.102643
KLD+10mmd_rbf_sum-1.0,0.000208,0.031234,0.289387,0.417703,0.552355,0.223627,0.482512,0.096065,0.368296,0.085383,0.354068
KLD+10mmd_rbf_sum-4.0,0.000026,0.000214,0.000440,0.000795,0.001014,0.000483,0.000998,0.000227,0.000654,0.000221,0.000599
KLD+10mmd_rbf_sum-7.0,0.000065,0.000355,0.000373,0.000282,0.000261,0.000355,0.000388,0.000321,0.000418,0.000319,0.000414
KLD+10mmd_rbf_sum-10.0,0.000160,0.000773,0.000199,0.000268,0.000135,0.000428,0.000159,0.000552,0.000159,0.000573,0.000158
RCL+KLD+10mmd_rbf_sum-1.0,0.001490,0.082718,0.105517,0.050380,0.040164,0.091135,0.066160,0.117163,0.087242,0.119185,0.090627
RCL+KLD+10mmd_rbf_sum-4.0,0.000329,0.034562,0.067208,0.027718,0.024224,0.049511,0.041961,0.060420,0.057276,0.060573,0.059766
RCL+KLD+10mmd_rbf_sum-7.0,0.000640,0.035586,0.064212,0.027110,0.023314,0.048262,0.040166,0.058884,0.054732,0.059071,0.057102
RCL+KLD+10mmd_rbf_sum-10.0,0.000661,0.035751,0.063968,0.027074,0.023233,0.048269,0.039978,0.059073,0.054419,0.059298,0.056769


Sample moments difference,1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
RCL+KLD,0.000019,0.770563,0.000853,2.917218,0.126729,15.633909,3.054554,113.467560,58.950279
KLD+10mmd_rbf_sum-1.0,20.284157,356.403259,1533.719727,839561.750000,17920936.000000,8440222720.000000,862659608576.000000,198080922124288.000000,32197533719592960.000000
KLD+10mmd_rbf_sum-4.0,0.630842,15.102108,546.374329,25518.638672,1375699.000000,82093128.000000,5259374080.000000,354405941248.000000,24773008556032.000000
KLD+10mmd_rbf_sum-7.0,0.088031,1.234200,16.415110,289.410889,5785.325684,128126.804688,3033622.500000,75163040.000000,1921357568.000000
KLD+10mmd_rbf_sum-10.0,0.021947,0.432012,0.671402,135.737228,60.152779,66401.703125,117885.445312,40079340.000000,183739376.000000
RCL+KLD+10mmd_rbf_sum-1.0,0.014806,0.742639,0.002661,2.889102,0.120739,15.601157,3.047363,113.428673,58.950527
RCL+KLD+10mmd_rbf_sum-4.0,0.027091,0.646232,0.001760,2.721235,0.119890,15.247086,3.016007,112.499191,58.696777
RCL+KLD+10mmd_rbf_sum-7.0,0.013260,0.642399,0.002815,2.724850,0.123359,15.269400,3.052516,112.610321,58.931690
RCL+KLD+10mmd_rbf_sum-10.0,0.002671,0.639389,0.006053,2.715738,0.146970,15.240191,3.128498,112.506477,59.242435


"Sample moments, coloured by difference",1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
GP,0.011229,1.011781,-0.001477,3.096120,-0.130892,15.854786,-3.070973,113.843201,-59.012207
RCL+KLD,0.011248,0.241218,-0.000624,0.178902,-0.004164,0.220877,-0.016419,0.375641,-0.061928
KLD+10mmd_rbf_sum-1.0,-20.272928,357.415039,-1533.721191,839564.875000,17920936.000000,8440222720.000000,862659608576.000000,198080922124288.000000,32197533719592960.000000
KLD+10mmd_rbf_sum-4.0,-0.619614,16.113890,-546.375793,25521.734375,-1375699.125000,82093144.000000,-5259374080.000000,354405941248.000000,-24773008556032.000000
KLD+10mmd_rbf_sum-7.0,0.099260,2.245982,16.413633,292.507019,5785.194824,128142.656250,3033619.500000,75163152.000000,1921357568.000000
KLD+10mmd_rbf_sum-10.0,-0.010719,1.443793,-0.672879,138.833344,-60.283672,66417.554688,117882.375000,40079452.000000,183739312.000000
RCL+KLD+10mmd_rbf_sum-1.0,-0.003577,0.269142,-0.004138,0.207017,-0.010153,0.253629,-0.023611,0.414526,-0.061679
RCL+KLD+10mmd_rbf_sum-4.0,-0.015863,0.365549,-0.003237,0.374884,-0.011002,0.607700,-0.054966,1.344010,-0.315429
RCL+KLD+10mmd_rbf_sum-7.0,-0.002031,0.369382,-0.004292,0.371270,-0.007533,0.585387,-0.018457,1.232878,-0.080519


In [7]:
parameter_experiment = "exp1"
number = "1" # disambiguate argument files

show_loss_scores(16, parameter_experiment, number)

#plot_training_histories(16, experiment, number, num_cols=4, num_rows=3, backfill="align_right")
#plot_trained_draws(16, experiment, number, num_cols=4, num_rows=3, backfill="align_right")

#show_loss_scores_mat(16, experiment, number)

None

In [24]:
parameter_experiment = "exp2"
number = "1" # disambiguate argument files

show_loss_scores(16, parameter_experiment, number)
# plot_training_histories(16, parameter_experiment, number, backfill="align_right")
# plot_trained_draws(16, parameter_experiment, number,  backfill="align_right", include_standard_vae=True)

Frobenius norms,1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
"0.1RCL+10mmd_rqk_sum-1,0.1+KLD",8.1e+00,1.1e+01,8.5e+00,1.0e+01,8.4e+00,9.5e+00,9.9e+00,1.1e+01,1.3e+01
"0.1RCL+10mmd_rqk_sum-2,0.1+KLD",6.7e+00,1.0e+01,7.2e+00,9.1e+00,6.9e+00,7.9e+00,7.7e+00,8.8e+00,9.5e+00
"0.1RCL+10mmd_rqk_sum-4,0.1+KLD",7.6e+00,1.0e+01,8.1e+00,9.7e+00,8.3e+00,9.7e+00,1.0e+01,1.1e+01,1.2e+01
"0.1RCL+10mmd_rqk_sum-6,0.1+KLD",8.8e+00,1.0e+01,8.9e+00,1.0e+01,1.0e+01,1.1e+01,1.2e+01,1.3e+01,1.3e+01
"0.1RCL+10mmd_rqk_sum-8,0.1+KLD",8.0e+00,1.0e+01,9.2e+00,1.0e+01,1.0e+01,1.1e+01,1.2e+01,1.2e+01,1.3e+01
"0.1RCL+10mmd_rqk_sum-1,1+KLD",8.5e+00,1.4e+01,1.0e+01,1.4e+01,1.1e+01,1.3e+01,1.2e+01,1.3e+01,1.4e+01
"0.1RCL+10mmd_rqk_sum-2,1+KLD",1.0e+01,1.2e+01,1.2e+01,1.4e+01,1.4e+01,1.5e+01,1.4e+01,1.5e+01,1.5e+01
"0.1RCL+10mmd_rqk_sum-4,1+KLD",1.0e+01,1.2e+01,1.3e+01,1.4e+01,1.4e+01,1.4e+01,1.4e+01,1.4e+01,1.4e+01
"0.1RCL+10mmd_rqk_sum-6,1+KLD",1.1e+01,1.1e+01,1.2e+01,1.3e+01,1.2e+01,1.2e+01,1.2e+01,1.3e+01,1.3e+01


MMD scores,mmd_rbf_sum-1.0,mmd_rbf_sum-4.0,mmd_rbf_sum-10.0,"mmd_rqk_sum-4.0,0.25","mmd_rqk_sum-8.0,0.25","mmd_rqk_sum-4.0,1","mmd_rqk_sum-8.0,1","mmd_rqk_sum-4.0,10","mmd_rqk_sum-8.0,10","mmd_rqk_sum-4.0,100","mmd_rqk_sum-8.0,100"
Loss functions,,,,,,,,,,,
"0.1RCL+10mmd_rqk_sum-1,0.1+KLD",4.4e-05,3.4e-03,1.3e-02,4.6e-03,4.5e-03,7.6e-03,8.2e-03,7.8e-03,1.2e-02,7.5e-03,1.2e-02
"0.1RCL+10mmd_rqk_sum-2,0.1+KLD",7.0e-05,3.1e-03,1.2e-02,4.3e-03,4.3e-03,7.0e-03,7.8e-03,7.1e-03,1.1e-02,6.8e-03,1.1e-02
"0.1RCL+10mmd_rqk_sum-4,0.1+KLD",1.2e-04,3.9e-03,1.1e-02,4.2e-03,4.0e-03,7.0e-03,7.3e-03,7.4e-03,1.0e-02,7.3e-03,1.1e-02
"0.1RCL+10mmd_rqk_sum-6,0.1+KLD",1.3e-04,4.5e-03,1.3e-02,5.1e-03,4.8e-03,8.5e-03,8.7e-03,9.0e-03,1.2e-02,8.8e-03,1.3e-02
"0.1RCL+10mmd_rqk_sum-8,0.1+KLD",1.3e-04,4.3e-03,1.3e-02,4.9e-03,4.7e-03,8.2e-03,8.6e-03,8.6e-03,1.2e-02,8.3e-03,1.3e-02
"0.1RCL+10mmd_rqk_sum-1,1+KLD",3.6e-05,3.6e-03,1.5e-02,5.4e-03,5.4e-03,8.8e-03,9.9e-03,8.7e-03,1.4e-02,8.4e-03,1.5e-02
"0.1RCL+10mmd_rqk_sum-2,1+KLD",3.8e-05,2.8e-03,1.1e-02,4.1e-03,4.0e-03,6.6e-03,7.4e-03,6.6e-03,1.0e-02,6.3e-03,1.1e-02
"0.1RCL+10mmd_rqk_sum-4,1+KLD",1.1e-04,3.9e-03,1.2e-02,4.7e-03,4.5e-03,7.8e-03,8.2e-03,8.2e-03,1.1e-02,7.9e-03,1.2e-02
"0.1RCL+10mmd_rqk_sum-6,1+KLD",1.5e-04,4.5e-03,1.3e-02,5.0e-03,4.8e-03,8.4e-03,8.6e-03,9.0e-03,1.2e-02,8.8e-03,1.3e-02


Sample moments (normal),1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
"0.1RCL+10mmd_rqk_sum-1,0.1+KLD",1.7e-02,6.5e-01,2.5e-02,1.2e+00,1.2e-01,3.5e+00,5.7e-01,1.4e+01,3.2e+00
"0.1RCL+10mmd_rqk_sum-2,0.1+KLD",-3.9e-02,6.6e-01,1.1e-02,1.2e+00,5.8e-02,3.6e+00,2.5e-01,1.4e+01,1.0e+00
"0.1RCL+10mmd_rqk_sum-4,0.1+KLD",2.8e-02,6.7e-01,7.5e-03,1.3e+00,3.0e-02,4.2e+00,2.5e-01,1.9e+01,2.7e+00
"0.1RCL+10mmd_rqk_sum-6,0.1+KLD",-1.9e-03,6.4e-01,-1.3e-02,1.2e+00,-3.2e-02,3.6e+00,1.2e-01,1.5e+01,3.4e+00
"0.1RCL+10mmd_rqk_sum-8,0.1+KLD",-2.5e-02,6.5e-01,-8.2e-04,1.2e+00,-4.5e-02,3.6e+00,-5.5e-01,1.5e+01,-5.4e+00
"0.1RCL+10mmd_rqk_sum-1,1+KLD",1.7e-03,6.2e-01,-5.1e-03,1.1e+00,-2.5e-02,2.8e+00,-1.1e-01,1.0e+01,-6.0e-01
"0.1RCL+10mmd_rqk_sum-2,1+KLD",-8.9e-03,6.7e-01,3.7e-03,1.3e+00,4.6e-02,4.1e+00,4.6e-01,1.8e+01,4.2e+00
"0.1RCL+10mmd_rqk_sum-4,1+KLD",-6.8e-04,6.7e-01,-7.3e-03,1.3e+00,-7.5e-02,4.6e+00,-7.2e-01,2.2e+01,-7.0e+00
"0.1RCL+10mmd_rqk_sum-6,1+KLD",-3.1e-03,6.6e-01,3.4e-03,1.3e+00,3.0e-02,4.4e+00,3.4e-01,2.1e+01,4.4e+00


Sample moments (ranked_colour),1,2,3,4,5,6,7,8,9
Loss functions,,,,,,,,,
GP,1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-1,0.1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-2,0.1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-4,0.1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-6,0.1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-8,0.1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-1,1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-2,1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01
"0.1RCL+10mmd_rqk_sum-4,1+KLD",1.1e-02,1.0e+00,-1.5e-03,3.1e+00,-1.3e-01,1.6e+01,-3.1e+00,1.1e+02,-5.9e+01


In [ ]:
parameter_experiment = "exp3"
number = "1" # disambiguate argument files

show_loss_scores(16, parameter_experiment, number)
plot_training_histories(16, parameter_experiment, number, backfill="align_right")
plot_trained_draws(16, parameter_experiment, number,  backfill="align_right", include_standard_vae=True)